# Cleaning data

Using the data dictionary from Kaggle as reference: [http://jse.amstat.org/v19n3/decock/DataDocumentation.txt](http://jse.amstat.org/v19n3/decock/DataDocumentation.txt)

In [9]:
# Import libraries here.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import pickle
%run ../assets/eda.py
pd.set_option('display.max_columns', None)

### 1. Read data

In [10]:
train = pd.read_csv('../datasets/train.csv')

Clean column names

In [11]:
# remove space in column names
train.columns = train.columns.str.replace(' ', '_')

# lowercase column names
train.columns = train.columns.str.lower()

### 2. Cleaning missing values

First let's look at the missing values in each column

In [12]:
train.shape

(2051, 81)

In [13]:
# look at which columns have the most missing data
# eda is a custom function I wrote to return the number of null values in each column
# and the data types for each column as a pandas data frame
# source for the code in assets/eda.py
eda(train)

,,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,utilities,lot_config,land_slope,neighborhood,condition_1,condition_2,bldg_type,house_style,overall_qual,overall_cond,year_built,year_remod/add,roof_style,roof_matl,exterior_1st,exterior_2nd,mas_vnr_type,mas_vnr_area,exter_qual,exter_cond,foundation,bsmt_qual,bsmt_cond,bsmt_exposure,bsmtfin_type_1,bsmtfin_sf_1,bsmtfin_type_2,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,heating,heating_qc,central_air,electrical,1st_flr_sf,2nd_flr_sf,low_qual_fin_sf,gr_liv_area,bsmt_full_bath,bsmt_half_bath,full_bath,half_bath,bedroom_abvgr,kitchen_abvgr,kitchen_qual,totrms_abvgrd,functional,fireplaces,fireplace_qu,garage_type,garage_yr_blt,garage_finish,garage_cars,garage_area,garage_qual,garage_cond,paved_drive,wood_deck_sf,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,saleprice
nulls,0,0,0,0,0,330,0,0,1911,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,22,22,0,0,0,55,55,58,55,1,56,1,1,1,0,0,0,0,0,0,0,0,2,2,0,0,0,0,0,0,0,0,1000,113,114,114,1,1,114,114,0,0,0,0,0,0,0,2042,1651,1986,0,0,0,0,0
datatypes,0,int64,int64,int64,object,float64,int64,object,object,object,object,object,object,object,object,object,object,object,object,int64,int64,int64,int64,object,object,object,object,object,float64,object,object,object,object,object,object,object,float64,object,float64,float64,float64,object,object,object,object,int64,int64,int64,int64,float64,float64,int64,int64,int64,int64,object,int64,object,int64,object,object,float64,object,float64,float64,object,object,object,int64,int64,int64,int64,int64,int64,object,object,object,int64,int64,int64,object,int64


__Fill misc_feature, alley, and fence missing values with "None"__

According to the data dictionary, NA values in misc features, alley, and fence means that there is none.

In [14]:
# fill in NA values
train["misc_feature"].fillna("None", inplace = True)
train["alley"].fillna("None", inplace = True)
train["fence"].fillna("None", inplace = True)

__Create a boolean column for has pool vs. no pool__

Since most of the rows have no pool, I think the quality grade is not as meaningful as "pool v. no pool."

In [15]:
# create a new boolean column for houses with pool v. no pool
train["has_pool"] = np.where(train["pool_qc"].notnull(), 1, 0)

# check if that gave me the same number of rows as the null values
# train[train["pool"]==0] has 2042 rows

Now I'm going to drop the rest of the pool columns.

In [16]:
# drop pool area and pool qc columns
train.drop(columns = ["pool_area", "pool_qc"], inplace = True)

__Create a boolean column for has remodel vs. no remodel__

The "year_remod/add" column is the remodel date, which is the same as the construction date if there were no remodeling or additions. I'll create a "has_remodel" column for houses that have had a remodel, by seeing if the "year_built" date is different from the "year_remod/add" date.


In [17]:
# create a new boolean column if the house has been remodeled vs. no remodel
train["has_remodel"] = np.where(train["year_built"] != train["year_remod/add"], 1, 0)

__Look at fireplace quality values__

Let's check if a null value in fireplace quality means that there is no fireplace. We can do this by taking a look at the fireplaces column.

In [18]:
# check the number of rows that have '0' in the fireplaces column
len(train[train["fireplaces"]==0])

1000

There are 1000 rows with no fireplace, which matches the number of missing rows in fireplace quality. I'll keep the missing values in fireplace quality as nulls.

__Look at garage variables__

According to the data dictionary, a NA in garage type variable means that there's no garage.

Garage Type (Nominal): Garage location
		
       2Types	More than one type of garage
       Attchd	Attached to home
       Basment	Basement Garage
       BuiltIn	Built-In (Garage part of house - typically has room above garage)
       CarPort	Car Port
       Detchd	Detached from home
       NA	No Garage

|column|NA's|
|---|---|
|garage_finish|114|
|garage_qual  |114|
|garage_cond  |114|
|garage_type  |113|
|garage_area  |1|
|garage_cars  |1|

Since there are 113 NA values in garage_type, and 114 in the other associated garage columns, this means that there's one true NA value.


In [19]:
# find the rows that do not have a garage
no_garage = train["garage_type"].isnull()  & train["garage_finish"].isnull() & train["garage_qual"].isnull()  & train["garage_cond"].isnull()  == True

In [20]:
# replace NA values with "None" across these rows
train.loc[no_garage, train.columns[train.columns.str.contains("garage")]] = train.loc[no_garage, train.columns[train.columns.str.contains("garage")]].fillna("None")

### 3. Change some ordinal values into numeric

__Look at ordinal definitions in our data set__

I'll convert columns with this ordinal scale to a numeric.

|Code|Definition|Numeric|
|:---|:---|:---|
|N|	No|0|
|Y|	Yes|1|

|Code|Definition|Numeric|
|:---|:---|:---|
|Ex|	Excellent|5|
|Gd|	Good|4|
|TA|	Average/Typical|3|
|Fa|	Fair|2|
|Po|	Poor|1|


|Code|Definition|Numeric|
|:---|:---|:---|
|Ex|	Excellent|4|
|Gd|	Good|3|
|TA|	Average/Typical|2|
|Fa|	Fair|1|

|Code|Definition|Numeric|
|:---|:---|:---|
|GLQ|	Good Living Quarters|5|
|ALQ|	Average Living Quarters|4|
|BLQ|	Below Average Living Quarters|3|
|Rec|	Average Rec Room|2|
|LwQ|	Low Quality|1|
|Unf|	Unfinshed|0|

In [21]:
# create a conversion dictionary
# this code was adapted from this stack overflow answer: https://stackoverflow.com/questions/21818886/changing-ordinal-character-data-to-numeric-data-with-pandas
conv_dict_2={
    "Y": 1,
    "N": 0
}

conv_dict_5 = {
    "Ex" : int(5),
    "Gd" : int(4),
    "TA" : int(3),
    "Fa" : int(2),
    "Po" : int(1),
}


conv_dict_4 = {
    "Ex" : int(4),
    "Gd" : int(3),
    "TA" : int(2),
    "Fa" : int(1)
}

conv_dict_6 ={
    "GLQ" : 5,
    "ALQ" : 4,
    "BLQ" : 3,
    "Rec" : 2,
    "LwQ" : 1,
    "Unf" : 0,
}

In [22]:
# use this to check how many columns have the same keys train.loc[:, train.nunique() == 5]
# create a custom function that takes in the columns we want to convert, dataframe, and dictionary
def replace_ordinal(columns, df, dictionary):
    for column in columns:
        df[column] = df[column].apply(dictionary.get)

In [23]:
# convert the columns with "Ex", "Gd", "TA", "Fa", and "Po" values to a numeric
replace_ordinal(["exter_qual",
                 "exter_cond",
                 "bsmt_qual", 
                 "bsmt_cond",
                 "heating_qc",
                 "kitchen_qual",
                 "garage_qual",
                 "garage_cond", 
                 "fireplace_qu"], train, conv_dict_5)

In [24]:
# fill NA's with 0 for basment quality
train["bsmt_qual"].fillna(0, inplace = True)

In [25]:
# fill NA's with 0 for garage quality
train["garage_qual"].fillna(0, inplace = True)

In [26]:
# fill NA's with 0 for garage cars
train["garage_cars"].fillna(0, inplace = True)

In [27]:
# convert the columns with "GLQ", "ALQ", "BLQ", "Rec", "LwQ", "Unf"  values to a numeric
replace_ordinal(["bsmtfin_type_1", "bsmtfin_type_2"], train, conv_dict_6)

In [28]:
# convert the columns with "N", "Y"  values to a numeric
replace_ordinal(["central_air", "paved_drive"], train, conv_dict_2)

### 4. Create dummy variables for neighborhood price ranges

Create a new variable that calculates price per square foot.

In [29]:
train["price_per_sqft"] = train["saleprice"]/train["gr_liv_area"]

In [30]:
# create a custom function that categorizes sale prices as 'high' 'medium' and 'low' in comparison to the mean saleprice of the data
def price_range(price):
    if price < (train["price_per_sqft"].mean() - np.std(train["price_per_sqft"]) ):
        return "low" # if price is below one std from mean, classify as low
    if price > (train["price_per_sqft"].mean() + np.std(train["price_per_sqft"]) ):
        return "high" # if price is one std abrove from mean, classify as high
    else:
        return "medium"

Create a new column for neighborhood_price to categorize how high/med/low the price is.

In [31]:
# return categorical variables for each price per sqft
train["neighborhood_price"] = train["price_per_sqft"].apply(price_range)

Create dummy variables

In [32]:
train["neighborhood_price"].value_counts()

medium    1447
low        310
high       294
Name: neighborhood_price, dtype: int64

In [33]:
# create dummy variables
train = pd.get_dummies(train, columns = ["neighborhood_price"])

In [34]:
# remove the medium column
train.drop("neighborhood_price_medium", axis=1, inplace = True)

__Let's create a dictionary for neighborhood and price range so we can replicate this in the Kaggle test data set__

Create a dataframe that just has the average saleprice for each neighborhood

In [35]:
# get average price per square foot per neighborhood
avg_neighborhood_prices = pd.DataFrame(train.groupby("neighborhood")["price_per_sqft"].mean())

Create a column that categorizes the saleprice as "high", "medium" and "low"

In [36]:
# classify each neighborhood price per square foot as "high" "medium" and "low"
avg_neighborhood_prices["range"] = avg_neighborhood_prices["price_per_sqft"].apply(price_range)

Convert the dataframe to a dictionary.
This dictionary will be used to categorize each neighborhood by its sale price in Kaggle test data set.

In [37]:
# convert df to dictionary
avg_neighborhood_prices[["range"]].to_dict()

{'range': {'Blmngtn': 'medium',
  'Blueste': 'medium',
  'BrDale': 'medium',
  'BrkSide': 'medium',
  'ClearCr': 'medium',
  'CollgCr': 'medium',
  'Crawfor': 'medium',
  'Edwards': 'medium',
  'Gilbert': 'medium',
  'Greens': 'medium',
  'GrnHill': 'high',
  'IDOTRR': 'low',
  'Landmrk': 'medium',
  'MeadowV': 'medium',
  'Mitchel': 'medium',
  'NAmes': 'medium',
  'NPkVill': 'medium',
  'NWAmes': 'medium',
  'NoRidge': 'medium',
  'NridgHt': 'high',
  'OldTown': 'medium',
  'SWISU': 'medium',
  'Sawyer': 'medium',
  'SawyerW': 'medium',
  'Somerst': 'medium',
  'StoneBr': 'high',
  'Timber': 'medium',
  'Veenker': 'medium'}}

### 5. Create a variable for housing stock per neighborhood

Create a new variable to see how much housing is available per neighborhood.

|Number of houses sold|Numeric|
|:---|:---|
|>200|5|
|100-200|4|
|50-100|3|
|25-50|2|
|0-25|1|

Create a dictionary to calculate the numeric value for number of houses sold.

In [38]:
train["neighborhood"].value_counts()

NAmes      310
CollgCr    180
OldTown    163
Edwards    143
Somerst    130
NridgHt    122
Gilbert    116
Sawyer     111
SawyerW     87
NWAmes      87
Mitchel     82
BrkSide     76
Crawfor     71
IDOTRR      69
Timber      48
NoRidge     48
StoneBr     38
SWISU       32
ClearCr     27
MeadowV     24
Blmngtn     22
BrDale      19
NPkVill     17
Veenker     17
Blueste      6
Greens       3
GrnHill      2
Landmrk      1
Name: neighborhood, dtype: int64

In [39]:
housing_stock = {
    "NAmes": 5,
    "CollgCr": 4,
    "OldTown": 4,
    "Edwards": 4,
    "Somerst": 4,
    "NridgHt": 4,
    "Gilbert": 4,
    "Sawyer": 4,
    "NWAmes": 3,
    "SawyerW": 3,
    "Mitchel": 3,
    "BrkSide": 3,
    "Crawfor": 3,
    "IDOTRR": 3,
    "NoRidge": 2,
    "Timber": 2,
    "StoneBr": 2,
    "SWISU": 2,
    "ClearCr": 2,
    "MeadowV": 1,
    "Blmngtn": 1,
    "BrDale": 1,
    "NPkVill": 1,
    "Veenker": 1,
    "Blueste": 1,
    "Greens": 1,
    "GrnHill": 1,
    "Landmrk": 1
}

In [40]:
# categorize neighborhood sale prices
train["housing_stock"] = train["neighborhood"].apply(housing_stock.get)

I'm going to create a pickle of this training clean set for EDA, before creating the dummy variables.

In [41]:
pickle.dump(train, open("../assets/train_clean_eda.pkl", "wb"))

### 6. Create dummy variables for zone

In [42]:
# start by looking at the counts for each
train["ms_zoning"].value_counts()

RL         1598
RM          316
FV          101
C (all)      19
RH           14
A (agr)       2
I (all)       1
Name: ms_zoning, dtype: int64

I'm going to create an "other" category for the zones with less than 100 rows. 

In [43]:
# create function to read through the values for each type
def format_zone(zone):
    if zone == "RL":
        return zone
    if zone == "RM":
        return zone
    if zone == "FV" :
        return zone
    else:
        return "Other"

In [44]:
# apply to ms_zone column
train["ms_zoning"] = train["ms_zoning"].apply(format_zone)

Create dummy variables for zone

In [45]:
train = pd.get_dummies(train, columns = ["ms_zoning"])

In [46]:
# remove the other column to set as default
train.drop("ms_zoning_Other", axis=1, inplace = True)

### 7. Create dummy variables for condition

In [47]:
# start by looking at the counts for each
train["condition_1"].value_counts()

Norm      1767
Feedr      109
Artery      70
RRAn        36
PosN        27
RRAe        21
PosA        12
RRNn         6
RRNe         3
Name: condition_1, dtype: int64

In [48]:
# create dummy variables
train = pd.get_dummies(train, columns = ["condition_1"])

In [49]:
# remove the normal column to set as default
train.drop("condition_1_Norm", axis=1, inplace = True)

### 8. Create dummy variables for garage type

In my research, garages that are attached to the are more valuable. So I will create an "other" category for either no garage or ones that are detached/carport.

In [50]:
# create function to read through the values for each type
def format_garage(garage):
    if garage == "Attchd":
        return garage
    if garage == "BuiltIn":
        return garage
    if garage == "Basment" :
        return garage
    else:
        return "Other"

In [51]:
# apply to garage_type column
train["garage_type"] = train["garage_type"].apply(format_garage)

Create dummy variables for garage type

In [52]:
train = pd.get_dummies(train, columns = ["garage_type"])

In [53]:
# remove the other column to set as default
train.drop("garage_type_Other", axis=1, inplace = True)

Create an interaction variable for garage quality * number of cars. Assuming that the more cars the garage fits, the better quality.

In [54]:
train["garage_qual_cars"] = train["garage_qual"] * train["garage_cars"]

### 9. Create a column for bathroom to bedroom ratio

In [55]:
train["bed_bath_ratio"] = train["bedroom_abvgr"] / (train["full_bath"] + train["half_bath"])

In [56]:
train["bed_bath_ratio"].fillna(0, inplace = True)

In [57]:
train["bed_bath_ratio"].replace(np.inf, 0, inplace = True)

# Pickle clean data for modeling

In [58]:
pickle.dump(train, open("../assets/train_clean_model.pkl", "wb"))